<a href="https://www.kaggle.com/jagritnokwal/ds-lab-eval?scriptVersionId=90306727" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import cupy
import nltk
import cudf as cd
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from cuml.linear_model import LogisticRegression
from cuml.feature_extraction.text import TfidfVectorizer
from cuml.metrics import confusion_matrix 
from sklearn.metrics import classification_report
from matplotlib import pyplot as plt
import seaborn as sns
import re

In [ ]:
train_data=cd.read_csv('/kaggle/input/amazon-pet-product-reviews-classification/train.csv',index_col='id').fillna()
valid_data=cd.read_csv('/kaggle/input/amazon-pet-product-reviews-classification/valid.csv',index_col='id').fillna()
test_data=cd.read_csv('/kaggle/input/amazon-pet-product-reviews-classification/test.csv',index_col='id').fillna()

In [ ]:
print(train_data.shape,test_data.shape)
train_data.head()

In [ ]:
from tqdm import tqdm
tqdm.pandas()

In [ ]:
sns.countplot(train_data['label'].to_array())
plt.title('Target distribution');

In [ ]:
def features(df):
    df=df.copy().to_pandas()
    stops = stopwords.words('english')
    stemmer = PorterStemmer()
    print(type(df['text']))
    # removing special characters
    df['prepared_text'] = df['text'].apply(lambda text: re.sub('[^A-Za-z]', ' ', text))
    # transform text to lowercase
    df['prepared_text'] = df['prepared_text'].str.lower()
    # tokenize the texts
    df['prepared_text'] = df['prepared_text'].apply(lambda text: word_tokenize(text))
    # removing stopwords
    df['prepared_text'] = df['prepared_text'].apply(lambda words: [word for word in words if word not in stops])
    # stemming
    df['prepared_text'] = df['prepared_text'].apply(lambda words: [stemmer.stem(word) for word in words])
    # join prepared+text to use as corpus
    df['joined_prepared_text'] = df['prepared_text'].apply(lambda words: " ".join(words))

    return cd.from_pandas(df)

In [ ]:
nltk.download('wordnet')


In [ ]:
train=features(train_data)
# train_data['text'].apply(lambda text: re.sub('[^A-Za-z]', ' ', text))
train.head()

In [ ]:
test=features(test_data)
test.head()

In [ ]:
td = TfidfVectorizer(max_features=10000)


In [ ]:
x = td.fit_transform(train['joined_prepared_text'])
x.shape

In [ ]:
labels_to_ids = {}
ids_to_labels = {}
for i, label in enumerate(sorted(train_data['label'].to_pandas().unique())):
    labels_to_ids[label] = i
    ids_to_labels[i] = label
    
labels_to_ids, ids_to_labels

In [ ]:
y = train['label'].map(labels_to_ids).values
y

In [ ]:
clf=LogisticRegression(C=5e1, solver='qn',max_iter=400).fit(x,y)


In [ ]:
y_pred = clf.predict(x)


In [ ]:
labels = list(ids_to_labels.values())


In [ ]:
print(classification_report(y.get(), y_pred.get(), target_names=labels))


In [ ]:
val = features(valid_data)


In [ ]:
x_val = td.transform(val['joined_prepared_text'])
x_val.shape

In [ ]:
y_true = val['label'].map(labels_to_ids).values
y_pred = clf.predict(x_val)

In [ ]:
print(classification_report(y_true.get(), y_pred.get(), target_names=labels))


In [ ]:
y_pred = cd.read_csv('/kaggle/input/amazon-pet-product-reviews-classification/sample_submission.csv',index_col='id')


In [ ]:
x_test = td.transform(test['joined_prepared_text'])
x_test.shape

In [ ]:
y_predicted = clf.predict(x_test)


In [ ]:
y_predicted.shape


In [ ]:
type(y_predicted)
# y_pred=cd.DataFrame()
y_pred.index_col='id'
y_pred['label']=y_predicted
# y_pred['id']=sample_submission['id']
y_pred['label']=y_pred['label'].map(ids_to_labels)

# y_predicted

In [ ]:
y_pred.to_csv('test_pred_df.csv', index_label='id', header=True)

In [ ]:
y_pred